In [1]:
import pandas as pd
import os

# Definisikan path atau lokasi kedua file
processed_dir = '../data/processed'
file_a_path = os.path.join(processed_dir, 'test_clean.csv')
file_b_path = os.path.join(processed_dir, 'datatest_revisi.csv')

def compare_csv_files(file_a, file_b, key_column):
    """
    Membandingkan dua file CSV dan mencetak perbedaannya.
    :param file_a: Path ke file CSV pertama.
    :param file_b: Path ke file CSV kedua.
    :param key_column: Nama kolom yang akan digunakan sebagai kunci unik untuk perbandingan.
    """
    try:
        # Baca kedua file ke dalam DataFrame
        df_a = pd.read_csv(file_a)
        df_b = pd.read_csv(file_b)

        print(f"✅ Berhasil membaca '{os.path.basename(file_a)}' dengan {len(df_a)} baris.")
        print(f"✅ Berhasil membaca '{os.path.basename(file_b)}' dengan {len(df_b)} baris.")
        
        # Mengubah nama kolom video di DataFrame B agar tidak sama saat digabungkan
        df_b_renamed = df_b.rename(columns={'video': 'video_b'})

        # Gabungkan kedua DataFrame berdasarkan kolom kunci ('id')
        merged_df = pd.merge(df_a, df_b_renamed, on=key_column, how='outer', indicator=True)
        
        # --- Bagian 1: Temukan baris yang hanya ada di satu file ---
        print("\n--- BARIS YANG BERBEDA ---")
        
        # Baris yang hanya ada di file A
        unique_to_a = merged_df[merged_df['_merge'] == 'left_only']
        if not unique_to_a.empty:
            print(f"⚠️ Baris yang hanya ada di '{os.path.basename(file_a)}':")
            print(unique_to_a.drop(columns=['_merge', 'video_b']))
        else:
            print(f"✅ Tidak ada baris unik di '{os.path.basename(file_a)}'.")

        # Baris yang hanya ada di file B
        unique_to_b = merged_df[merged_df['_merge'] == 'right_only']
        if not unique_to_b.empty:
            print(f"\n⚠️ Baris yang hanya ada di '{os.path.basename(file_b)}':")
            print(unique_to_b.drop(columns=['_merge', 'video']))
        else:
            print(f"\n✅ Tidak ada baris unik di '{os.path.basename(file_b)}'.")

        # --- Bagian 2: Temukan baris yang ada di kedua file, tapi isinya berbeda ---
        print("\n--- ISI BARIS YANG BERBEDA (Video URL) ---")

        # Hanya ambil baris yang ada di kedua file
        both_present = merged_df[merged_df['_merge'] == 'both'].copy()
        
        # Bandingkan kolom 'video' dan 'video_b'
        diff_video = both_present[both_present['video'] != both_present['video_b']]

        if not diff_video.empty:
            print(f"⚠️ Video URL berbeda untuk ID berikut:")
            print(diff_video[[key_column, 'video', 'video_b']])
        else:
            print("✅ Tidak ada perbedaan video URL untuk baris yang sama.")

    except FileNotFoundError as e:
        print(f"❌ Error: Satu atau lebih file tidak ditemukan. Pastikan path-nya benar: {e}")
    except KeyError:
        print(f"❌ Error: Kolom '{key_column}' tidak ditemukan di salah satu file. Pastikan nama kolom kuncinya sudah benar.")
    except Exception as e:
        print(f"❌ Terjadi kesalahan: {e}")

# Jalankan fungsi perbandingan
# Menggunakan 'id' sebagai kolom kunci karena kedua file memilikinya
compare_csv_files(file_a_path, file_b_path, 'id')

✅ Berhasil membaca 'test_clean.csv' dengan 200 baris.
✅ Berhasil membaca 'datatest_revisi.csv' dengan 200 baris.

--- BARIS YANG BERBEDA ---
✅ Tidak ada baris unik di 'test_clean.csv'.

✅ Tidak ada baris unik di 'datatest_revisi.csv'.

--- ISI BARIS YANG BERBEDA (Video URL) ---
⚠️ Video URL berbeda untuk ID berikut:
      id                                              video  \
17    18  https://drive.google.com/file/d/12YBpn9zWc_3Ff...   
25    26  https://drive.google.com/file/d/12YBpn9zWc_3Ff...   
26    27  https://drive.google.com/file/d/12YBpn9zWc_3Ff...   
27    28  https://drive.google.com/file/d/12YBpn9zWc_3Ff...   
106  107  https://drive.google.com/drive/u/7/folders/1gt...   

                                               video_b  
17   https://www.instagram.com/reel/DMz13fQzZsN/?ig...  
25   https://www.instagram.com/reel/DKQ8RCqypek/?ig...  
26   https://www.instagram.com/reel/DI24B5NS1Ng/?ig...  
27   https://www.instagram.com/reel/DMzlY-uxiog/?ig...  
106  https://www.i